Мы будем парсить данные о погоде для гонок, в которых в результате первого парсинга мы не получили информации

Будем парсить данные с сайта http://f1-data.com/script/menu/race.php. Для каждой гонки на этом сайте существуют отдельные страницы, например: https://f1-data.com/script/db/race/classify.php?race=103
Ссылки на них отличаются только номером гонки, причем номера гонок начинаются с 1 и идут в порядке от самой старой к самой новой.

На странице гонок есть таблица, в которой содержится информация о погоде, а также информация о том, сухая или мокрая трасса в процессе гонки. Мы будем параметр track записывать в столбец weather_main, так как там как раз содержится информация о том, какие условия гонки ('wet', 'dry', 'changing'), а общие данные о погоды записывать в столбец weather.

Для этого используем таблицу new, в которой к таблице races была добавлена колонка newid, показывающая номер гонки в порядке от самой ранней к самой поздней.

Таблица new была загружена на github

In [23]:
import pandas as pd
import requests
import numpy as np
from bs4 import BeautifulSoup

In [103]:
race = pd.read_csv('https://raw.githubusercontent.com/pavleke/project-formula-1/main/races.csv')
new = race.sort_values(by='date')
new['newid'] = np.arange(1, len(new)+1)
new

,raceId,year,round,circuitId,name,date,time,url,fp1_date,fp1_time,fp2_date,fp2_time,fp3_date,fp3_time,quali_date,quali_time,sprint_date,sprint_time,newid
832,833,1950,1,9,British Grand Prix,1950-05-13,\N,http://en.wikipedia.org/wiki/1950_British_Gran...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N,1
833,834,1950,2,6,Monaco Grand Prix,1950-05-21,\N,http://en.wikipedia.org/wiki/1950_Monaco_Grand...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N,2
834,835,1950,3,19,Indianapolis 500,1950-05-30,\N,http://en.wikipedia.org/wiki/1950_Indianapolis...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N,3
835,836,1950,4,66,Swiss Grand Prix,1950-06-04,\N,http://en.wikipedia.org/wiki/1950_Swiss_Grand_...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N,4
836,837,1950,5,13,Belgian Grand Prix,1950-06-18,\N,http://en.wikipedia.org/wiki/1950_Belgian_Gran...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1097,1116,2023,19,69,United States Grand Prix,2023-10-22,19:00:00,https://en.wikipedia.org/wiki/2023_United_Stat...,2023-10-20,17:30:00,2023-10-21,18:00:00,\N,\N,2023-10-20,21:00:00,2023-10-21,22:00:00,1098
1098,1117,2023,20,32,Mexico City Grand Prix,2023-10-29,20:00:00,https://en.wikipedia.org/wiki/2023_Mexico_City...,2023-10-27,18:30:00,2023-10-27,22:00:00,2023-10-28,17:30:00,2023-10-28,21:00:00,\N,\N,1099
1099,1118,2023,21,18,São Paulo Grand Prix,2023-11-05,17:00:00,https://en.wikipedia.org/wiki/2023_S%C3%A3o_Pa...,2023-11-03,14:30:00,2023-11-04,14:30:00,\N,\N,2023-11-03,18:00:00,2023-11-04,18:30:00,1100
1100,1119,2023,22,80,Las Vegas Grand Prix,2023-11-19,06:00:00,https://en.wikipedia.org/wiki/2023_Las_Vegas_G...,2023-11-17,04:30:00,2023-11-17,08:00:00,2023-11-18,04:30:00,2023-11-18,08:00:00,\N,\N,1101


In [73]:
weather = pd.read_csv('https://raw.githubusercontent.com/pavleke/project-formula-1/main/weather.csv')

In [74]:
weather_new_id = new.merge(weather, how='left', right_index=True, left_index=True)
weather_new_id #Таблица с newid (по порядку от самой старой к самой новой гонке и weather)
weather_newid = weather_new_id[['weather', 'weather_main', 'newid', 'raceId', 'name', 'date']]
weather_newid

,weather,weather_main,newid,raceId,name,date
832,"Sunny, mild, dry.",dry,1,833,British Grand Prix,1950-05-13
833,NaN,NaN,2,834,Monaco Grand Prix,1950-05-21
834,Rainy,wet,3,835,Indianapolis 500,1950-05-30
835,"Warm, dry and sunny",dry,4,836,Swiss Grand Prix,1950-06-04
836,"Warm, dry and sunny",dry,5,837,Belgian Grand Prix,1950-06-18
...,...,...,...,...,...,...
1097,NaN,NaN,1098,1116,United States Grand Prix,2023-10-22
1098,NaN,NaN,1099,1117,Mexico City Grand Prix,2023-10-29
1099,NaN,NaN,1100,1118,São Paulo Grand Prix,2023-11-05
1100,NaN,NaN,1101,1119,Las Vegas Grand Prix,2023-11-19


Создадим список индексов newid гонок, для которых требуется дополнительный парсинг погоды

In [75]:
weather_needed = weather_newid.loc[(weather_newid['date'] < '2023-05-15') & (weather_newid['weather_main'].fillna('nan') == 'nan')]
needed_id = list(weather_needed['newid']) #Получим список id (по порядку) гонок, для которых нам нужно провести дополнительный парсинг погоды
needed_id

[2,
 43,
 45,
 48,
 53,
 55,
 57,
 58,
 60,
 62,
 64,
 65,
 66,
 67,
 68,
 70,
 71,
 72,
 73,
 74,
 77,
 85,
 86,
 87,
 88,
 89,
 92,
 122,
 131,
 132,
 133,
 137,
 139,
 141,
 151,
 152,
 153,
 154,
 155,
 156,
 161,
 180,
 188,
 191,
 192,
 193,
 197,
 202,
 204,
 206,
 209,
 211,
 213,
 214,
 216,
 221,
 224,
 226,
 227,
 228,
 232,
 233,
 238,
 239,
 240,
 241,
 242,
 243,
 244,
 245,
 247,
 248,
 249,
 252,
 256,
 257,
 263,
 268,
 269,
 272,
 275,
 277,
 285,
 287,
 289,
 303,
 319,
 321,
 333,
 336,
 361,
 367,
 371,
 377,
 379,
 382,
 391,
 392,
 398,
 407,
 418,
 431,
 444,
 497,
 507,
 540,
 543,
 755]

Сделаем дополнительный парсинг погоды для гонок с найденными индексами, в процессе этого будем делить track на 'wet', 'dry' и 'changing'. Weather_main считается changing, когда в гонке в некоторые периоды мокрая трасса, а в некоторые - сухая.

In [43]:
weather_all_needed = []
tracks_all_needed = []
all_links = ["http://f1-data.com/script/db/race/classify.php?race=1"]
for i in needed_id:    
    html = all_links[-1]
    response = requests.get(url=all_links[-1])    
    soup = BeautifulSoup(response.content, 'html.parser')
    table = soup.find("table").get_text().split('\n')
    weather_found = table[-4]
    track_found = table[-3]
    if (track_found.find('wet') != -1) and (track_found.find('dry') != -1):
        track = 'changing'
    else:
        track = track_found
    weather_all_needed.append(weather_found)
    tracks_all_needed.append(track)
    all_links.append('http://f1-data.com/script/db/race/classify.php?race=' + str(i+1)) 
tracks_all_needed, weather_all_needed

(['dry',
  'changing',
  'dry',
  'dry',
  'dry',
  'dry',
  'dry',
  'dry',
  'dry',
  'dry',
  'dry',
  'dry',
  'dry',
  'dry',
  'dry',
  'dry',
  'dry',
  'dry',
  'dry',
  'dry',
  'dry',
  'dry',
  'dry',
  'dry',
  'dry',
  'dry',
  'dry',
  'dry',
  'dry',
  'dry',
  'dry',
  'wet',
  'dry',
  'changing',
  'dry',
  'dry',
  'dry',
  'dry',
  'dry',
  'dry',
  'dry',
  'dry',
  'dry',
  'dry',
  'dry',
  'dry',
  'dry',
  'dry',
  'dry',
  'dry',
  'wet',
  'dry',
  'wet',
  'dry',
  'dry',
  'dry',
  'dry',
  'dry',
  'dry',
  'dry',
  'dry',
  'dry',
  'changing',
  'changing',
  'dry',
  'dry',
  'dry',
  'dry',
  'dry',
  'dry',
  'dry',
  'dry',
  'dry',
  'dry',
  'dry',
  'dry',
  'changing',
  'dry',
  'dry',
  'dry',
  'dry',
  'dry',
  'dry',
  'dry',
  'dry',
  'dry',
  'dry',
  'dry',
  'dry',
  'changing',
  'dry',
  'dry',
  'dry',
  'dry',
  'changing',
  'dry',
  'dry',
  'dry',
  'dry',
  'dry',
  'dry',
  'dry',
  'dry',
  'dry',
  'dry',
  'dry',
  'dry',
  

Совместим всю информацию, которую мы спарсили в таблицу weather_dop

In [76]:
weather_dop = pd.DataFrame({'newid': needed_id, 'weather_main': tracks_all_needed, 'weather': weather_all_needed})
weather_dop

,newid,weather_main,weather
0,2,dry,fine
1,43,changing,"fine, rain later"
2,45,dry,fine
3,48,dry,"fine, rain later"
4,53,dry,fine partly cloudy
...,...,...,...
103,497,dry,fine
104,507,dry,fine
105,540,dry,fine
106,543,dry,fine partly cloudy


Совместим таблицы weather_newid и weather_dop для того, чтобы в будущем получить полную информацию о погоде
Оставим в таблице weather_merged только гонки, в которых изначально не было информации о погоде

In [89]:
weather_merged = weather_newid.merge(weather_dop, how='left', on='newid')
weather_merged.drop(['weather_x', 'weather_main_x'], axis=1, inplace=True)
weather_merged.rename(columns={'weather_main_y':'weather_main', 'weather_y': 'weather'}, inplace=True)
weather_merged.dropna(inplace=True)
weather_merged

,newid,raceId,name,date,weather_main,weather
1,2,834,Monaco Grand Prix,1950-05-21,dry,fine
42,43,793,Monaco Grand Prix,1955-05-22,changing,"fine, rain later"
44,45,795,Belgian Grand Prix,1955-06-05,dry,fine
47,48,798,Italian Grand Prix,1955-09-11,dry,"fine, rain later"
52,53,788,French Grand Prix,1956-07-01,dry,fine partly cloudy
...,...,...,...,...,...,...
496,497,333,Portuguese Grand Prix,1990-09-23,dry,fine
506,507,311,French Grand Prix,1991-07-07,dry,fine
539,540,280,French Grand Prix,1993-07-04,dry,fine
542,543,283,Hungarian Grand Prix,1993-08-15,dry,fine partly cloudy


Удалим из таблицы weather_newid все строчки с NaN и назовем новую таблицу weather_del

In [92]:
weather_del = weather_newid.dropna()
weather_del

,weather,weather_main,newid,raceId,name,date
832,"Sunny, mild, dry.",dry,1,833,British Grand Prix,1950-05-13
834,Rainy,wet,3,835,Indianapolis 500,1950-05-30
835,"Warm, dry and sunny",dry,4,836,Swiss Grand Prix,1950-06-04
836,"Warm, dry and sunny",dry,5,837,Belgian Grand Prix,1950-06-18
837,Hot and sunny,dry,6,838,French Grand Prix,1950-07-02
...,...,...,...,...,...,...
1079,Clear,dry,1080,1098,Bahrain Grand Prix,2023-03-05
1080,Clear,dry,1081,1099,Saudi Arabian Grand Prix,2023-03-19
1081,Sunny,dry,1082,1100,Australian Grand Prix,2023-04-02
1082,Partly cloudy,dry,1083,1101,Azerbaijan Grand Prix,2023-04-30


Совместим таблицы weather_del и weather_merged для того, чтобы получить таблицу с полной информацией о погоде

In [97]:
weather_full = pd.concat([weather_del, weather_merged], axis=0, join='outer', ignore_index=True)
weather_full

,weather,weather_main,newid,raceId,name,date
0,"Sunny, mild, dry.",dry,1,833,British Grand Prix,1950-05-13
1,Rainy,wet,3,835,Indianapolis 500,1950-05-30
2,"Warm, dry and sunny",dry,4,836,Swiss Grand Prix,1950-06-04
3,"Warm, dry and sunny",dry,5,837,Belgian Grand Prix,1950-06-18
4,Hot and sunny,dry,6,838,French Grand Prix,1950-07-02
...,...,...,...,...,...,...
1079,fine,dry,497,333,Portuguese Grand Prix,1990-09-23
1080,fine,dry,507,311,French Grand Prix,1991-07-07
1081,fine,dry,540,280,French Grand Prix,1993-07-04
1082,fine partly cloudy,dry,543,283,Hungarian Grand Prix,1993-08-15


In [100]:
weather_full.sort_values('raceId', inplace=True)

Сохраним файл с полной информацией о погоде на github

In [102]:
weather_full.to_csv('weather_full')